# Jetpack Compose - Repozytorium - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu **Repozytorium** w aplikacji.

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeWR3YThvcTgxcDVhbzl2MzFja2hmcG9mMmc0d2s1aXpicDA5eHlsaSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/b2E5YdNOMy3IC3fVU8/giphy.gif" width="200" />

**Repozytorium** jest warstwą pośredniczącą między danymi a resztą aplikacji, zapewniającymi dostęp do źródeł danych (np. bazy danych, zdalne API, odczyt z pliku) oraz operacje na danych. W kontekście wzorca `MVVM`, repozytorium pomaga w oddzieleniu logiki biznesowej od interakcji z danymi, co ułatwia testowanie, utrzymanie i rozszerzanie aplikacji. 
- Separacja odpowiedzialności - Izoluje logikę dostępu do danych od logiki biznesowej i ui. To pozwala na łatwiejsze zarządzanie i utrzymanie kodu, a także ułatwia współpracę między różnymi zespołami programistycznymi.
- Testowalność - Zapewnia łatwość testowania, ponieważ zapewnia abstrakcję nad danymi. Dzięki temu możemy tworzyć testy jednostkowe i testy integracyjne, które pozwalają nam weryfikować poprawność funkcjonalności bez konieczności dostępu do rzeczywistych źródeł danych.
- Łatwa wymiana źródeł danych - Dzięki repozytorium możemy zmieniać źródło danych bez wprowadzania zmian w innych częściach aplikacji. Na przykład, jeśli nasza aplikacja korzysta z lokalnej bazy danych, ale chcemy w przyszłości przejść na zdalne API, możemy to zrobić bez konieczności modyfikowania `ViewModel`'ów i warstwy ui.
- Obsługa błędów i odzyskiwanie - Repozytoria mogą obsługiwać błędy związane z danymi, takie jak problemy z siecią czy błędy związane z bazą danych.
- Optymalizacja dostępu do danych - Repozytoria mogą wprowadzać mechanizmy optymalizacji dostępu do danych, np. *cache'owanie* wyników zapytań czy ograniczenie ilości komunikacji sieciowej. To może przyspieszyć działanie aplikacji i zmniejszyć zużycie zasobów urządzenia. (*cachowanie* pojawi się w ostatnim module zajęć)
- Logika dostępu do danych - jeżeli mamy kilka źródeł danych, logikę dostępu (dostęp do API co określony czas, na żądanie użytkownika, w przeciwnym wypadku wczytanie danych z lokalnej bazy) możemy umieścić w repozytorium, pozostawiając `ViewModel` z jednym źródłem danych

Stosowanie repozytorium nie jest obowiązkowe we wzorcu MVVM (nie jest jego częścią), ale jest jednym z wielu sposobów na zorganizowanie kodu w bardziej czytelny sposób, co ułatwia pracę nad większymi i bardziej skomplikowanymi projektami. 

<img src="https://media.licdn.com/dms/image/D5612AQFCAOUrDob0aw/article-inline_image-shrink_1500_2232/0/1686677040244?e=1696464000&v=beta&t=ZL1tZWV3kJMxsXuV-5bWswoQfC3Q0CHavGwjUcaqTTM" width="600" />

Do wykorzystania repozytorium nie są wymagane żadne dodatkowe zależności w projekcie, dodajmy viewmodel

```kotlin
    implementation "androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1"
    implementation "androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1"
    implementation "androidx.lifecycle:lifecycle-runtime-compose:2.6.1"
```

Nasza aplikacja będzie wyświetlała listę użytkowników, którą wygenerujemy (*dummy data*). Dodajmy model, oraz dane.

In [ ]:
data class User(val firstName: String, val lastName: String)

In [ ]:
object DataProvider {

    private val firstNames = listOf(
        "Adam", "Ewa", "Jan", "Anna", "Piotr", "Maria", "Tomasz", "Małgorzata", "Krzysztof", "Alicja",
        "Andrzej", "Joanna", "Michał", "Barbara", "Kamil", "Magdalena", "Robert", "Monika", "Mateusz", "Natalia"
    )

    private val lastNames = listOf(
        "Nowak", "Kowalski", "Wiśniewski", "Wójcik", "Kowalczyk", "Kamiński", "Lewandowski", "Zieliński", "Szymański",
        "Woźniak", "Dąbrowski", "Kozłowski", "Jankowski", "Mazur", "Kwiatkowski", "Krawczyk", "Piotrowski", "Grabowski",
        "Nowakowski", "Pawłowski"
    )

    val users = (0..40).map { User(firstNames.random(), lastNames.random()) }
}

Dodajmy repozytorium, jest to zwykła klasa.

In [ ]:
class UserRepository {
    suspend fun getUsers(): List<User> {
        delay(700L)
        return DataProvider.users
    }
}

Ten kod reprezentuje klasę o nazwie `UserRepository`, która jest odpowiedzialna za dostarczanie listy użytkowników (`User`) z źródła danych.

- `suspend fun getUsers(): List<User> { ... }` - Funkcja zwracająca listę użytkowników. Powodem oznaczenia tej funkcji jako `suspend` jest wykorzystanie funkcji `delay()` w celu symulacji opóźnienia.
- `delay(700L)` Powoduje wstrzymanie wykonywania kodu przez określony czas (700 milisekund).

W następnym kroku dodajmy `ViewModel`

In [ ]:
class UserViewModel : ViewModel() {

    private val userRepository = UserRepository()

    private val _usersList = MutableStateFlow<List<User>>(emptyList())
    val usersList: StateFlow<List<User>> get() = _usersList

    init {
        loadUsers()
    }

    private fun loadUsers() {
        viewModelScope.launch {
            _usersList.value = userRepository.getUsers()
        }
    }
}

`UserViewModel` służy jako warstwa pośrednicząca między ui a repozytorium. Po utworzeniu obiektu, inicjalizuje się funkcję `loadUsers()`, która asynchronicznie pobiera listę użytkowników z repozytorium i przechowuje je w `_usersList`.

- `init { ... }` - Blok inicjalizacyjny, który zostanie wywołany podczas tworzenia obiektu `UserViewModel`. W bloku inicjalizacyjnym wywoływana jest funkcja `loadUsers()`, która ma za zadanie załadować listę użytkowników z repozytorium i umieścić je w `_usersList`.
- `viewModelScope.launch { ... }` - `viewModelScope` jest dostarczany przez `ViewModel` i automatycznie zarządza cyklem życia Coroutine w zależności od cyklu życia `ViewModel`. `userRepository.getUsers()` wewnątrz bloku `launch` asynchronicznie pobiera listę użytkowników.

Utwórzmy `@Composable` renderujący ekran naszej aplikacji.

In [ ]:
@Composable
fun MainScreen() {

    val viewModel: UserViewModel = viewModel()
    val users by viewModel.usersList.collectAsStateWithLifecycle()

    LazyColumn() {
        items(users.size) {
            Text(
                text = "${users[it].firstName} ${users[it].lastName}",
                fontSize = 32.sp,
                textAlign = TextAlign.Center,
                modifier = Modifier
                    .fillMaxWidth()
                    .padding(2.dp)
            )
        }
    }
}

Na koniec wywołajmy funkcję w głównej aktywności

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            RepositoryBasicsComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    MainScreen()
                }
            }
        }
    }
}

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExeWR3YThvcTgxcDVhbzl2MzFja2hmcG9mMmc0d2s1aXpicDA5eHlsaSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/b2E5YdNOMy3IC3fVU8/giphy.gif" width="200" />